In [2]:
import cv2
import numpy as np

In [4]:
# Load YOLOv2 model
net = cv2.dnn.readNet("yolov2.cfg", "yolov2.weights")

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov2.cfg in function 'cv::dnn::dnn4_v20230620::readNetFromDarknet'


In [21]:
classes = []
with open("coco.names", "r") as f:
    classes = f.read().strip().split('\n')


In [22]:
table_tennis_bat_class_name = "table tennis bat"
table_tennis_ball_class_name = "table tennis ball"

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Read a frame from the camera

    # Get frame dimensions
    height, width = frame.shape[:2]

    # Create a blob from the frame and pass it through the network
    blob = cv2.dnn.blobFromImage(frame, 1.0/255.0, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    output_layer_names = net.getUnconnectedOutLayersNames()
    layer_outputs = net.forward(output_layer_names)

    class_ids = []
    confidences = []
    boxes = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Adjust confidence threshold as needed
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Calculate coordinates for the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)  # Apply non-maximum suppression

    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(class_ids), 3))

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y + 30), font, 3, color, 3)

    # Display the frame with bounding boxes
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()